<a target="_blank" href="https://colab.research.google.com/github/Jasonchen0317/CSGY-6613-Assignment/blob/main/assignment-3/assignment-3.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00


In [3]:
!pip install --upgrade -q git+https://github.com/keras-team/keras-cv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 12.2 MB/s eta 0:00:00


In [4]:
from pytube import YouTube
import cv2
import os
from google.colab import output

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import optimizers
import keras_cv
import numpy as np
from keras_cv import bounding_box
import os
import resource
from keras_cv import visualization
import tqdm


Using TensorFlow backend


In [ ]:
def download_video(video_list):
    for i in range(len(video_list)):
        #get link
        link = video_list[i]
        yt = YouTube(link)
        yt.bypass_age_gate()
        #Download video
        filename = "%s.mp4" % str(i)
        yt.streams.filter(res="720p").first().download(filename=filename, output_path='video')

In [ ]:
if not os.path.exists('video'):
   os.makedirs('video')

video_list = [
    'https://www.youtube.com/watch?v=DhmZ6W1UAv4',
    'https://www.youtube.com/watch?v=YrydHPwRelI'
]

In [ ]:
download_video(video_list)

In [ ]:
v_list = os.listdir('video')

In [ ]:
len(v_list)

2

In [ ]:
!rm -rf frames/

In [ ]:
for i in range(len(v_list)):
  vid = i
  capture = cv2.VideoCapture(f'video/{vid}.mp4')
  length = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
  print( length )

4941
15409


In [ ]:
if not os.path.exists('frames'):
   os.makedirs('frames')

for i in range(len(v_list)):
  vid = i
  capture = cv2.VideoCapture(f'video/{vid}.mp4')
  frameNr = 0
  success, frame = capture.read()
  while (success):
      cv2.imwrite(f'frames/video_{vid}_frame_{frameNr}.jpg', frame)
      print(f'Saved video {vid} frame {frameNr}!')
      if frameNr%10 == 0:
        output.clear()

      success, frame = capture.read()
      frameNr = frameNr+1

  capture.release()


Saved video 1 frame 15401!
Saved video 1 frame 15402!
Saved video 1 frame 15403!
Saved video 1 frame 15404!
Saved video 1 frame 15405!
Saved video 1 frame 15406!
Saved video 1 frame 15407!
Saved video 1 frame 15408!


In [ ]:
class_ids = [
    "Drone",
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))


In [5]:
def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
    inputs = next(iter(inputs.take(1)))
    images, bounding_boxes = inputs["images"], inputs["bounding_boxes"]
    visualization.plot_bounding_box_gallery(
        images,
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale=5,
        font_scale=0.7,
        bounding_box_format=bounding_box_format,
        class_mapping=class_mapping,
    )

def unpackage_raw_inputs(inputs, bounding_box_format):
    image = inputs["image"]
    boxes = keras_cv.bounding_box.convert_format(
        inputs["objects"]["bbox"],
        images=image,
        source="rel_yxyx",
        target=bounding_box_format,
    )
    bounding_boxes = {
        "classes": tf.cast(inputs["objects"]["label"], dtype=tf.float32),
        "boxes": tf.cast(boxes, dtype=tf.float32),
    }
    return {"images": tf.cast(image, tf.float32), "bounding_boxes": bounding_boxes}


def load_data(split, dataset, bounding_box_format):
    ds = tfds.load(dataset, split=split, with_info=False, shuffle_files=True)
    ds = ds.map(
        lambda x: unpackage_raw_inputs(x, bounding_box_format=bounding_box_format),
        num_parallel_calls=tf.data.AUTOTUNE,
    )
    return ds


In [ ]:
train_image = []
train_annotation = []

In [ ]:
import glob
from PIL import Image
image_file = glob.glob("C:/kaggle/archive(2)/drone_dataset_yolo/*.jpg")

In [6]:
for i in image_file:     #slicing for insufficient memory
    Load_image = Image.open(i).convert('RGB')
    nmpy_img = Load_image.resize((258,258))
    train_image.append(np.asarray(nmpy_img))

    seperate_path = i.split('.')
    seperate_path[1] = '.txt'

    with open(seperate_path[0]+seperate_path[1]) as f:
        lines = f.readlines()
        tmp_lst = lines[0].split(' ')
        #label = int(tmp_lst[0])
        startX = float(tmp_lst[1])
        startY = float(tmp_lst[2])
        endX = float(tmp_lst[3])
        endY = float(tmp_lst[4])
        train_annotation.append((startX, startY, endX, endY))
